# Preparing the walmart data. 

AIM: The notebook helps download walmart data and turn it to a format similar to M&S hierarchical data. [Link to the dataset](https://www.kaggle.com/competitions/m5-forecasting-accuracy)

<br></br>
<div style="text-align: center; line-height: 5; padding-top: 20px;  padding-bottom: 20px;">
  <img src="https://raw.githubusercontent.com/puneet-jain159/Image_dump/2a8b03eef9bd111b98b261846d57b72ce98fde38/walmart_data_description.png" alt='Push compute' height="1000" width="1600">
</div>

### Install Dependencies

In [0]:
%pip install kaggle

In [0]:
import pandas as pd
import numpy as np
import os
import mlflow

### Setup UC configurations and Kaggle credentials to get M5 dataset

In [0]:
config = mlflow.models.ModelConfig(development_config='config.yaml')
CATALOG = config.get('catalog')
SCHEMA = config.get('schema')
VOLUME = config.get('volume')

In [0]:
# Check if UC assets exists and create them if they do not
spark.sql(f"USE CATALOG {CATALOG}")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {SCHEMA}")
spark.sql(f"USE SCHEMA {SCHEMA}")
spark.sql(f"CREATE VOLUME IF NOT EXISTS {CATALOG}.{SCHEMA}.{VOLUME}")

### Read the downloaded data and analyize all the different files

In [0]:
sdf_calendar = spark.read.csv(f'/Volumes/{CATALOG}/{SCHEMA}/{VOLUME}/calendar.csv', header="True")
print(sdf_calendar.count())
sdf_calendar.display()

In [0]:
sdf_sales_train_evaluation = spark.read.csv(f'/Volumes/{CATALOG}/{SCHEMA}/{VOLUME}/sales_train_evaluation.csv', header="True")
print( sdf_sales_train_evaluation.count())
sdf_sales_train_evaluation.display()

In [0]:
sdf_sell_prices= spark.read.csv(f'/Volumes/{CATALOG}/{SCHEMA}/{VOLUME}/sell_prices.csv', header="True")
print(sdf_sell_prices.count())
sdf_sell_prices.display()

### Transformation and clean the data
We transform and melt the column into rows

In [0]:
ids =['id','item_id','dept_id','cat_id','store_id','state_id'] 
cols = [col for col in sdf_sales_train_evaluation.columns if col not in ids ] 

# pivot the data
sdf_sales_train_evaluation_pivot = (sdf_sales_train_evaluation.melt(
    ids=ids, values=cols,
    variableColumnName="date", valueColumnName="sale_quantity"))

sdf_sales_train_evaluation_pivot.write.mode("overwrite").format('delta').saveAsTable(config.get('eval_table'))

In [0]:
# merge to get date
sdf_calendar = sdf_calendar.withColumnRenamed("date", "date_time")
cond = [sdf_sales_train_evaluation_pivot.date == sdf_calendar.d]
sdf_sales_train_evaluation_pivot = sdf_sales_train_evaluation_pivot.join(
    sdf_calendar.select(["d", "wm_yr_wk", "date_time"]), cond, "left"
)

# merge to get sales price
cond = [
    sdf_sales_train_evaluation_pivot.wm_yr_wk == sdf_sell_prices.wm_yr_wk,
    sdf_sales_train_evaluation_pivot.item_id == sdf_sell_prices.item_id,
    sdf_sales_train_evaluation_pivot.store_id == sdf_sell_prices.store_id,
]
final_table = sdf_sales_train_evaluation_pivot.join(sdf_sell_prices, cond, "left").select(
    sdf_sales_train_evaluation_pivot.item_id,
    sdf_sales_train_evaluation_pivot.dept_id,
    sdf_sales_train_evaluation_pivot.cat_id,
    sdf_sales_train_evaluation_pivot.store_id,
    sdf_sales_train_evaluation_pivot.state_id,
    sdf_sales_train_evaluation_pivot.date_time,
    sdf_sell_prices.sell_price,
    sdf_sales_train_evaluation_pivot.sale_quantity,
)

# remove nulls sell_price and sales_quantity = 0
final_table = final_table.filter(final_table.sale_quantity > 0)

### Write out the final table as a Delta Table

In [0]:
final_table.write.mode("overwrite").format('delta').saveAsTable(config.get('final_table'))